In [189]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [190]:
import spacy

In [191]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [192]:
torch.cuda.is_available()

True

In [193]:
tweets = pd.read_csv('data/olid-training-v1.0.tsv', sep='\t')

In [194]:
tweets

,id,tweet,subtask_a,subtask_b,subtask_c
0,86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND
2,16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN
4,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN
...,...,...,...,...,...
13235,95338,@USER Sometimes I get strong vibes from people...,OFF,TIN,IND
13236,67210,Benidorm ✅ Creamfields ✅ Maga ✅ Not too sh...,NOT,NaN,NaN
13237,82921,@USER And why report this garbage. We don't g...,OFF,TIN,OTH
13238,27429,@USER Pussy,OFF,UNT,NaN


In [195]:
tweets = tweets[:5]

In [196]:
tweets = tweets[:5]

In [197]:
tweets

,id,tweet,subtask_a,subtask_b,subtask_c
0,86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND
2,16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN
4,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN


In [198]:
first = tweets.loc[0, 'tweet']

In [199]:
len(first)

71

In [200]:
nlp = spacy.load('en_core_web_sm')

In [201]:
# dodawanie kolumny z tokenami
tweets['tokens'] = tweets['tweet'].apply(nlp)

In [202]:
tweets

,id,tweet,subtask_a,subtask_b,subtask_c,tokens
0,86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN,"(@USER, She, should, ask, a, few, native, Amer..."
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND,"(@USER, @USER, Go, home, you, ’re, drunk, !, !..."
2,16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN,"(Amazon, is, investigating, Chinese, employees..."
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN,"(@USER, Someone, should'veTaken, "", this, piec..."
4,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN,"(@USER, @USER, Obama, wanted, liberals, &, amp..."


In [203]:
# dodawanie kolumny z lematami za pomoca funkcji lambda - przy tym usuwanie odwolan do wczesniejszych tweetow (zawiera @)
tweets['lemmas'] = tweets['tokens'].apply\
    (lambda list_tokens : [token.lemma_ for token in list_tokens if ('@' not in token.lemma_ and '#' not in token.lemma_ and not token.is_stop and not token.is_punct)])

In [204]:
tweets['length_lemmas'] = tweets['lemmas'].apply(lambda row: len(row))

In [205]:
tweets['length_lemmas']

0     3
1     9
2    19
3     5
4     7
Name: length_lemmas, dtype: int64

In [206]:
max_length_tweet = max(tweets['length_lemmas'])

In [207]:
def convert_words_to_numbers(lemmas_series):
    dict_ = dict()
    for row in lemmas_series:
        for lemma in row:
            if lemma not in dict_:
                dict_[lemma] = len(dict_) + 1

    return dict_

In [208]:
dict_lemmas = convert_words_to_numbers(tweets['lemmas'])

In [215]:
sorted(dict_lemmas)[:10]

['Amazon',
 'Americans',
 'CHINA',
 'KAG',
 'MAGA',
 'Obama',
 'Trump2020',
 'amp',
 'ask',
 'chinese']

In [210]:
def lemmas_to_numbers(row, max_list_lemmas, dict_):
    list_numbers = []
    for i in range(max_list_lemmas - len(row)):
        list_numbers.append(0)

    for lemma in row:
        list_numbers.append(dict_[lemma])
    return list_numbers

In [211]:
# zamiana lematu na liczbe ze slownika ktory odpowiada danemu slowu
tweets['numbers'] = tweets['lemmas'].apply(lambda row : lemmas_to_numbers(row, max_length_tweet, dict_lemmas))

In [212]:
tweets['lemmas']

0                             [ask, native, Americans]
1      [home, drunk, MAGA, Trump2020, 👊, 🇺, 🇸, 👊, url]
2    [Amazon, investigate, chinese, employee, sell,...
3            [should'vetaken, piece, shit, volcano, 😂]
4     [Obama, want, liberal, amp, illegal, red, state]
Name: lemmas, dtype: object

In [213]:
tweets.loc[0, 'numbers']

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 3]